# DevCloud to Local Migration
This tutorial explains the differences between running your IoT application on the DevCloud and in a local environment as well as how to migrate your DevCloud application to your local environment. Because everyone's application has unique requirements, it is intended as a guide and not the definitive reference for doing so.

For the purpose of this tutorial, we will use the Object Detection Demo: Safety Gear Detection from the [Advanced Sample Applications page](https://devcloud.intel.com/edge/advanced/sample_applications/) as our sample to migrate to a local machine. 

## OpenVINO version check:
You are currently using the latest development version of Intel® Distribution of OpenVINO™ Toolkit. Alternatively, you can open a version of this notebook for the Intel® Distribution of OpenVINO™ Toolkit LTS version by running the cell below and following the link it generates.

In [ ]:
from qarpo import displayMultiversionURL
import os
displayMultiversionURL(os.path.abspath(""), "DevCloud_to_Local_Migration.ipynb", "openvino-dev-latest", ['openvino-lts'])

## Understand what the DevCloud does/does not do
The Intel DevCloud for the Edge is a useful tool for quickly prototyping, developing and optimizing a new application or importing an existing application and then analyzing the application performance across multiple hardware environments. Since the hardware is in the cloud, it is not necessary to purchase specific hardware on which to run your application until you are closer to deploying it. The DevCloud environment uses consistent interfaces to the AI tools to facilitate being able to deploy on multiple types of hardware. An overview of the DevCloud for the Edge can be found at [here](https://devcloud.intel.com/edge/get_started/devcloud/)

### How components of the DevCloud translate to local environments

From the illustration below some components of the DevCloud are not present or different in a local execution environment. The circled numbers refer to the DevCloud graphic below. 
- The HTTP browser❶ access to Development Servers❷ is typically replaced by direct terminal access in your local environment
- Your local development machine replaces the Development Servers❷ and the Jupyter Notebook. 
- The Storage Server, accessed as `/data` within the Development Server environment, does not exist in your local environment. You will need to provide someplace in your local environment for this data.
- The Job Queue❸ is the means within the DevCloud to share access to limited hardware resources❹ in the cloud. Since your local deployment environment will typically have only one hardware resource and dedicated access, the queuing system is not required.
- The Inference Video/Image Output❺ will be stored directly to your local machine and the Results❻ will be viewed directly instead of via an HTTP browser.

<img src="https://devcloud.intel.com/edge/static/images/svg/Edge-howitworks_PN3.svg" />


## The DevCloud Tutorial Prerequisites

Although this tutorial references a specific demo in the DevCloud, the concepts and the steps will also be necessary for your own project migration.
### Execute the notebook
For this tutorial, you will need to have previously executed all of the cells in the Object Detection Demo: Safety Gear Detection notebook found on the [Advanced Sample Applications page](https://devcloud.intel.com/edge/advanced/sample_applications/) so that all the models have been downloaded, the IR files generated and optimized, and the data is in place. Just follow the steps in the notebook or select `Cell -> Run All` from the menu before continuing.
### Save and Checkpoint
When the execution is completed, select `File -> Save and Checkpoint` to ensure the notebook is saved in the completed state.
### Save/download the Notebook as Python source (if required)
If you have created your application code in a Python Notebook(s) and not as a separate executable .py file, you should download the notebook(s) as a Python (.py) file(s). Launch your notebook(s) and use the menu `File -> Download as -> Python (.py)` to get your notebook downloaded to a local Python file.
<img src="resources/screenshot-download-py.png" />

## Prerequisites and assumptions for local environment
Before you begin to migrate your application from the DevCloud to your local environment, there are a minimal set of requirements you need to have in place on your local machine. 

For this tutorial, we will be migrating to a local instance of Ubuntu 18.04 with Intel(R) OpenVINO(TM) toolkit 2019 R3 and its dependencies installed. The target environment will be the CPU of this machine.

### Installed matching version of OpenVINO and its prerequisites
To ensure a successful migration, validate that your installed local OpenVINO version matches the version you are using in the DevCloud. Run the command below to see which version of OpenVINO has been linked to `/opt/intel/openvino/`. You can run the same command in your local environment. If they do not match, download and install the matching version of OpenVINO in your local environment.

In [ ]:
!awk '/OpenVINO/{gsub("for.*$","");print;exit}' /opt/intel/openvino/licensing/OpenVINOsupport.txt

### Minimal text editing ability
A minimal editor installed in your local environment is required to allow you to modify your code and shell scripts for operation locally.


## Download the required files from your DevCloud Jupyter Notebook environment
The DevCloud Jupyter Notebook environment allows downloading single files at a time. This section will show what can be done to get multiple files at once and provides some commands to make this happen.

### Know which files are needed
It is unlikely that all of the files on the DevCloud are required in your local environment. To start determining where the needed files are you need to first know the base directory of your notebook. Insert the following cell into your project notebook and execute it. You can delete the cell after noting the directory path.

In [ ]:
# find my DevCloud notebook directory
!pwd

For this tutorial our results will look similar to:
<img src="resources/Screenshot-pwd-safety-gear.png" />

You will be downloading files from this directory and its subdirectories and possibly files from the `/data` directory depending on how your project is organized. For the Safety Gear Detection tutorial, we will need to retrieve the input movie files from `/data`.

<br><div class=tbd><small><b>TBD </b>Using the [nbzip notebook extension](https://github.com/data-8/nbzip) or a derivative would be the preferred method here for packaging and downloading a set of files. However, until this available in the DevCloud, the terminal instructions below may be used.</small></div>

### Terminal instructions
The Jupyter Notebook environment allows for downloading the notebook itself from the `File` menu. You can also use the file manager and select (checkmark) a single file at a time which enables the Download button. To download your entire Jupyter workspace, a whole directory, or multiple files, use the terminal as follows below.

1. Launch a Linux terminal by clicking `New` and then `Terminal`:
<img src="https://devcloud.intel.com/edge/static/images/png/edge-jupyter-new-terminal.png" />

#### Download your entire workspace:
This section is for reference only. If you are following the tutorial, skip to the next section [Download select directory](#Download-select-directory)

This section details how to archive your _entire_ DevCloud workspace and download it to your local machine.

The following assumes that we will create a tar-gzipped file called workspace.tar.gz containing all of the files from your workspace in your home directory. In the shell prompt, type or paste the statements that follow.

2. Use the terminal's command line to change the current working directory to the top level of your workspace. 

```bash
cd ~
```

3. Remove the previous archive, if it exists: 

```bash
rm -f ~/workspace.tar.gz*
```

4. Create a zipped archive of your workspace directory: 

```bash
tar -czvf ~/workspace.tar.gz ~/
```

<br><div class=note><i><b>Note: </b>Because you are creating the tar file in the same directory that is being processed, tar will warn you that a “file changed as we read it”. You can safely ignore this.</i></div>

5. Once the commands run successfully, return to the tab/window that shows the file manager view shown in step 1. In the file manager, select(check the box beside) workspace.tar.gz ①, then click Download ②. Your browser will download the workspace archive to a directory of your choosing -- typically your `~/Downloads` directory.

<img src="resources/screenshot-download-workspace.png" />

6. (Optional) Remove the archive file on the DevCloud that you just downloaded (it should still be selected) by clicking on the trashcan icon ③

<br><div class=warn><b>Warning: </b>Jupyter uploads and downloads are typically limited to 100MB due to limitations built into the dependent Tornado code. If your zip file is larger than 100MB, you may need to split it up into smaller files instead and download each of them, using the following commands instead of the above tar command:

```bash
tar -czvf - ~/ | split --bytes=100MB - ~/workspaces.tar.gz
```

This creates multiple files with the prefix 'workspaces.tar.gz' followed by 'aa', 'ab', etc. After downloading these to your local machine, you can re-combine them with:

```bash
cat workspaces.tar.gz* | tar -xvzf -
```
</i></div>

#### Download select directory

This section describes how to create an archive of a selected directory within your DevCloud workspace and download it to your local machine.

Follow step 1 above to launch a terminal.

The following assumes that we will create a tar-gzipped file called <directory_name>.tar.gz in your home directory containing all of the files from your selected project directory. In the shell prompt, type or paste the statements that follow.

2. Use the terminal's command line to change the current working directory to the project's base directory discovered with the `pwd` command above.

```bash
cd <directory_name>
``` 

<br><div class=note><i><b>Note: </b>The "/home/ . . . /" at the start of the directory path is discarded since that is unique for each DevCloud user.</i></div>

For the tutorial, use this command to get to our project base directory:

```bash
cd Reference-samples/iot-devcloud/python/safety-gear-detection-python
```

3. Remove the previous archive, if it exists: 

```bash
rm -f ~/safety-gear-detection-python.tar.gz
```

4. Create a zipped archive of your selected directory named <project_directory_name>.tar.gz in your home directory on the DevCloud: 

```bash
tar -cvzf ~/$(basename $(pwd)).tar.gz -C .. $(basename $(pwd))
```

5. Once the commands run successfully, return to the tab/window that shows the file manager view shown in step 1. In the file manager, select(check the box beside) safety-gear-detection-python.tar.gz ①, then click Download ②. Your browser will download the directory archive.
<img src="resources/screenshot-download-safety-gear.png" />


6. (Optional) Remove the archive file you just downloaded (it should still be selected) by clicking on the trashcan icon ③.

<br><div class=warn><b>Warning: </b>Jupyter uploads and downloads are typically limited to 100MB. If your zip file is larger than 100MB, you may need to split it up into smaller files instead and download each of them, using the following commands instead of the above tar command:

```bash
tar -C .. -czvf - $(basename $(pwd)) | split --bytes=100MB - ~/$(basename $(pwd)).tar.gz
```

This creates multiple files with the prefix '<project_directory_name>.tar.gz' followed by 'aa', 'ab', etc. After downloading these to your local machine, you can re-combine them with:

```bash
cat <project_directory_name>.tar.gz* | tar -xvzf -
```
</i></div>

##### Download /data files
A review of the `object_detection_job.sh` and the notebook file, `safety_gear_detection.py`, in the tutorial reveals references to files in `/data/`.

<img src="resources/screenshot-safety-gear-detection-py-data-1.png" />

Therefore we also need to download these files from the `/data/reference-sample-data/safety-gear-detection/` directory. To do so we execute the following in the Jupyter terminal:

```bash
cd ~
tar -cvzf safety-gear-detection-data.tar.gz -C /data/reference-sample-data/safety-gear-detection *
```

Return to the Jupyter file manager tab of your browser, select the `safety-gear-detection-data.tar.gz` file and download it to your local machine.

## Extract the archive
If your archive is of the entire workspace, you may want to first create a directory into which you will extract the archive:

```bash
mkdir ~/<my_local_workspace>

#goto this directory or wherever you want to extract your downloaded files
cd ~/<my_local_workspace>
```

For this tutorial we will extract the `safety-gear-detection-python.tar.gz` into our home directory and then extract the `safety-gear-detection-data.tar.gz` into a subdirectory of that project.

```bash
# go to our home directory
cd ~

# extract the previously downloaded project archive into this directory
tar -xvzf Downloads/safety-gear-detection-python.tar.gz

# create a subdirectory in the project for the data archive extraction
mkdir safety-gear-detection-python/data

# extract the /data files
tar -xvzf Downloads/safety-gear-detection-data.tar.gz -C safety-gear-detection-python/data
```

Verify that your project files and data have been successfully extracted:

```bash
ls -R ~/safety-gear-detection-python
```

## Prepare for local execution
### Local directory changes
The archive commands used above will include symbolic links that are no longer valid in the local environment. You can fix these, or just note and delete them so that they cannot be used. Run the following:

```bash
find -L ~/safety-gear-detection-python -type l -exec ls -l {} \;
```

In this case the above command will show the `Safety_Full_Hat_and_Vest.mp4` file as a link to the `/data` directory which is not valid. We can just remove this link and refer directly to the that file when needed. It is now located in the `~/safety-gear-detection-python/data/` directory from when we downloaded the files.

``` bash
rm ~/safety-gear-detection-python/Safety_Full_Hat_and_Vest.mp4
```


### Source changes
#### Parameter and variable localization
If the .py file you plan to execute does not include the ability to accept command line configuration parameters, you will likely want that flexibility. The `ArgumentParser` object from the [argparse.py](https://docs.python.org/3/library/argparse.html#) Python library makes this task easy. Determine which sources, outputs, paths, models, devices, plugins, data, maps, labels, output options, iterations, and optimizations you might want to make configurable from the command line, and implement parsing with appropriate default values. The default values used in the DevCloud, especially path-related variables may need to change to match your local environment. Some parameters that may have been useful on the DevCloud, such as devices, may be static in your local environment and extra code to handle certain device types may no longer be required.

For this tutorial, source references to files in `/data/.../<file>` will need to be changed to reference `data/<file>` or `~/safety-gear-detection-python/data/<file>`

<br><div class=note><i><b>Note: </b>As mentioned in Step 2 of the Safety Gear Object Detection notebook, one of the parameters that might change in the local environment is the use of a live camera feed instead of a video for input. This can be accomplished by changing the input file argument to `'cam'`</i></div>

#### Console print to logs
In the Jupyter Notebook environment, it was easy to print progress and debug information to the screen. In the local environment, this type of output is more appropriately captured in a log file. For this tutorial, this step has already been done in the `object_detection_demo-ssd_async.py` file. 

For your projects, you may need to import the `logging` library and initialize it at the beginning of your main function. You can find more information on using the library [here](https://docs.python.org/3/library/logging.html#).

```python
import logging as log
...
log.basicConfig(format="[ %(levelname)s ] %(message)s", level=log.INFO, stream=sys.stdout)
```
#### Job IDs in print or logs
Since your local environment does not contain the job queuing envionment of the DevCloud, references to job IDs in logs and print statements are not necessary. All references to job IDs should be removed from the code. However, for this tutorial, the `PBS_JOBID` environment variable is used in the code and we can easily set this variable and avoid having to make further code changes related to job IDs. The job ID is used for creating output file names. There are several other environment variables referenced that we will also set now for later use.

```bash
# define a job ID for ouput file names
export PBS_JOBID=01

# SKIPFRAME determines the processing of every SKIPFRAME frames
export SKIPFRAME=1

# RESOLUTION determines the scaled resolution of the output video
export RESOLUTION=0.5
```

#### Removing use of demoTools
Most of the tutorials in the DevCloud make use of the `demoTools` module to display progress bars for jobs that are initiated. This module is only available in the DevCloud environment. Another means of displaying progress updates would need to be created in your local environment if that is desirable. For the purpose of this tutorial we will just disable the use of this module. The implementation of a replacement for these tools is left as an exercise should it become necessary.

Using your favorite editor, make the following changes to the `object_detection_demo_ssd_async.py` file:

1. Make line 30 a comment by putting a "#" at the beginning of the line:
```python
# from demoTools.demoutils import progressUpdate
```
2. Similarly comment out lines 136 and 137 (these create file names that are only used by progressUpdate):
```python
# pre_infer_file = os.path.join(args.output_dir, 'pre_progress_'+job_id+'.txt')
# infer_file = os.path.join(args.output_dir, 'i_progress_'+job_id+'.txt')# from demoTools.demoutils import progressUpdate
```
3. Also comment out lines 169 and 170 (call to progressUpdate):
```python
# if id_%10 == 0: 
#    progressUpdate(pre_infer_file, time.time()-time_start, id_, video_len)
```
4. And finally, also comment lines 210 and 211 (also a call to progressUpdate):
```python
# if frame_count % 10 == 0: a
#    progressUpdate(infer_file, time.time()-infer_time_start, frame_count+1, video_le    n+1)
```

5. Save the file.

### Job files in the local environment
Job files that are useful in the DevCloud environment are not needed in the local environment. They are used for submitting a job to the various shared hardware instances on the DevCloud. However, you do need to understand what arguments the job files use, what parameters they supply and what function they provide in order to successfully execute your project locally.

Let us take a closer look at the `object_detection_job.sh` file. 

The first several lines only determine the name of the job file and set the base directory of where to run the job on the working directory on the DevCloud. Locally we already know where our working directory is.

<img src="resources/screenshot-job-01-06.png" />

The next section of the job file processes command line arguments.

<img src="resources/screenshot-job-09-34.png" />

- the -d argument stores the name of the device to use for inference as `DEVICE`
- the -f argument stores the base directory of the floating point model we are using in `FP_MODEL`. For this example it would be `models/mobilenet-ssd/FP32`.
- the -i argument specifies the input file or device as `INPUT_FILE`
- the -r argument specifies the results output directory as `RESULTS_OUTPUT`
- the -n argument specifies the number of inference requests as `NUM_INFER`

The next line specifies `NN_MODEL` as `mobilnet-ssd.xml` which is the model file name within the `FP_MODEL` directory.

<img src="resources/screenshot-job-35-36.png" />

The next few lines ensure the output directory is created.

<img src="resources/screenshot-job-37-40.png" />

The next IF block handles the condition for setting up the environment if the device we want to use is "HETERO:FPGA,CPU". You can take note of these lines if that is the device you intend to use locally. For this tutorial, we will use "CPU" as the device so we can ignore this section.

<img src="resources/screenshot-job-41-48.png" />

The next lines (49-58) are the main objective of the job file -- to run the object detection inference code on the input:

<img src="resources/screenshot-job-49-59.png" />

This is the command line we need to use in our local environment, first substituting appropriate paths, device, model, labels and number of inferences. For this tutorial, the command line is shown in the next step.


## Execute local application
Now we are ready to execute the inference code on the local environment using the information we gleaned above. Execute the following:

```bash
python3 object_detection_demo_ssd_async.py \
   -m models/mobilenet-ssd/FP32/mobilenet-ssd.xml \
   -i data/Safety_Full_Hat_and_Vest.mp4 \
   -o local-results \
   -d CPU \
   -ce /opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so \
   --labels data/labels.txt
```
You should observe the following output in your terminal:
```
[ INFO ] Initializing plugin for CPU device...
[ INFO ] Loading plugins for CPU device...
[ INFO ] Reading IR...
[ INFO ] Loading IR to the plugin...
[ INFO ] Starting inference in async mode, 2 requests in parallel...
[ INFO ] Starting inference in async mode...
[ INFO ] To switch between sync and async modes press Tab button
[ INFO ] To stop the sample execution press Esc button
[ INFO ] Processing done...
```
This will create two files in the `local-results` directory:  `output_01.txt` and `stats_01.txt`.

### Viewing output
We skipped the final section (lines 59-end) of the `object_detection_job.sh` file, but we return to it now to complete the tutorial. The last six lines of the file compile and execute the ROI_writer utility, which uses the inference output to render output video as an overlay on the source video. First compile the utility.
```bash
g++ -std=c++14 ROI_writer.cpp -o ROI_writer \
    -lopencv_core -lopencv_videoio -lopencv_imgproc \
    -lopencv_highgui  -fopenmp \
    -I/opt/intel/openvino/opencv/include/ \
    -L/opt/intel/openvino/opencv/lib/
```
We already defined SKIPFRAME and RESOLUTION above as environment variables. You can re-define those if needed or just put the new values on the command line below. The ROI_writer requires four arguments:
1. The path of the input video file used for inference, `data/Safety_Full_Hat_and_Vest.mp4`
2. The path of the directory where the inference output was written, `local-results`
3. Process every X frames: 1 = process all frames, 2 = every other frame, 3 = every third frame, etc. We previously set the SKIPFRAME environment variable to 1.
4. The scale ratio of the output frames (0.75, 0.5, 1). We previously set the RESOLUTION environment variable to 0.5. Higher resolutions will take longer to post process.

Therefore we can execute the writer as follows:
```bash
./ROI_writer data/Safety_Full_Hat_and_Vest.mp4 \
   local-results \
   $SKIPFRAME \
   $RESOLUTION
```
Results should indicate the post-processing time and place an `output_01.mp4` file in the `local-results` directory.
```
Video post-processing time: 18.9516 seconds
```

The resulting mp4 file can be viewed using [totem](https://help.gnome.org/users/totem/stable/totem-introduction.html.en), the default Linux video player (GNOME/GStreamer player) using the command:

```bash
totem local-results/output_01.mp4
```

### Validating performance
To validate the performance locally you need only look at the `local-results/stats_01.txt` file.
```
7.76 
1950 
```
The first line is the inference engine processing time in seconds. The second line is the inference engine frames per second value. Since the local environment is not comparing multiple inferencing engines, it is not necessary to plot multiple values for comparison.

You should compare your local results against those achieved in the DevCloud on an equivalent hardware target.

### Troubleshooting local execution issues

Although local execution issues are unique to each developer, probably the most common issues experienced are a result of mismatched versions of OpenVINO, Python and the supporting libraries. These would manifest as errors during Python execution or C++ compiles.

If your local execution environment requires the tool sets you have installed (assuming they are mismatched), you may have to modify code to conform to those libraries.

Other issues might be addressed within the Intel forum(s) associated with your edge device.

## Next Steps and Links to further reading
### Optimization
Generally it is expected that you use the DevCloud environment for optimizing your application before you port it back to your local environment. However, there may still be some localized tweaking you will want to implement. Rather than replicating multiple ideas for further optimization of your application, for considering latency versus batch size, and for other adjustments, the following links provide more information.

#### Intel Links

- [OpenVINO Model Optimizer Developer Guide](https://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)
- [OpenVINO Optimization Guide](https://docs.openvinotoolkit.org/latest/_docs_optimization_guide_dldt_optimization_guide.html) (this is a sub-link of the Model Optimizer Developer Guide)
- [Model Optimization Techniques](https://docs.openvinotoolkit.org/latest/_docs_MO_DG_prepare_model_Model_Optimization_Techniques.html) (this is a sub-link of the Model Optimizer Developer Guide)
(and further links accessible from these pages)
- [CPU Inference Performance Boost with “Throughput” Mode in the Intel® Distribution of OpenVINO™ Toolkit](https://www.intel.ai/cpu-inference-performance-boost-openvino/#gs.uqlebj)

#### Non-Intel Links
- [Optimizing neural networks for production with Intel’s OpenVINO](https://hackernoon.com/optimizing-neural-networks-for-production-with-intels-openvino-a7ee3a6883d) Cutting pre- and post-processing operations from TensorFlow graphs

### OpenVINO deployment
- [OpenVINO™ Deployment Manager Guide](https://docs.openvinotoolkit.org/latest/_docs_install_guides_deployment_manager_tool.html)